# Projeto 3 - Ciência dos Dados

#  Projeto UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



  ___
## Objetivo:

### O objetivo desse projeto é fazer um machine learning para poder prever qual lutador ganhará uma luta do UFC baseado no seu histórico. Para isso serão usados os dados de todas as lutas do UFC (mais de 5 mil) a fim de descobrir quais os fatores dos lutadores que impactam mais no resultado das lutas. 

[Database utilizado](https://www.kaggle.com/rajeevw/ufcdata#data.csv)

 ___
## Preparando o ambiente no jupyter:

### Imports:

In [1]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random
import statsmodels.api as sm
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### Trabalhando com os Excels:

In [2]:
data = pd.read_excel("data.xlsx")
preproc = pd.read_excel("preprocessed_data.xlsx")
fighter = pd.read_excel("raw_fighter_details.xlsx")
total = pd.read_excel("raw_total_fight_data.xlsx")
data2 = data
data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0


In [3]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [4]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

In [5]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### Blue = 0
#### Red = 1

In [6]:
data_heavy.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",0,0,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,Jerin Valel,2019,"Ottawa, Ontario, Canada",1,0,Heavyweight,3,0,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [7]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [8]:
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.0,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [9]:
data_heavy_util.dropna(inplace=True)
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.00,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.00,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.00,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.00,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
75,Alistair Overeem,Aleksei Oleinik,1,0,Heavyweight,5,0,2,0,4.25,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


#### Tirando os dados categóricos:

In [10]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_heavy_cat = data_heavy_util[categoricas].astype('category')
data_heavy_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

In [11]:
X = data_heavy_num.drop('Winner', axis=1)
y = data_heavy_num['Winner']

### Separando os dados em testes e treinamento

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [13]:

model = RandomForestClassifier(n_estimators=10000)

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
#Fatores que tem o maior peso na decisão da vitória
j=1
lista_j=list()
for i, f in sorted(list(zip(model.feature_importances_, X_train.columns)), reverse=True):
    print( j,'°:',f'{f}: {i}')
    a=str(j)+'°'
    lista_j.append(a)
    j+=1


1 °: R_avg_opp_TOTAL_STR_landed: 0.015668803235770738
2 °: R_avg_opp_CLINCH_landed: 0.01504517747141978
3 °: R_age: 0.01504369485885921
4 °: R_total_rounds_fought: 0.015023952358479446
5 °: B_avg_BODY_landed: 0.01380260122499282
6 °: B_avg_BODY_att: 0.013224129827308835
7 °: R_avg_opp_HEAD_landed: 0.01320113046082939
8 °: R_avg_opp_CLINCH_att: 0.01285749516478218
9 °: B_avg_HEAD_att: 0.012595021020686865
10 °: R_avg_opp_GROUND_landed: 0.012104874968346512
11 °: B_avg_TD_att: 0.011437434469863025
12 °: R_avg_CLINCH_att: 0.01137497761806011
13 °: B_avg_TOTAL_STR_att: 0.010766299173339088
14 °: B_avg_SIG_STR_att: 0.010601896847073336
15 °: R_avg_CLINCH_landed: 0.01060028374504822
16 °: R_losses: 0.010581294195177117
17 °: B_avg_opp_SIG_STR_pct: 0.010563984533764887
18 °: R_wins: 0.010540486276453884
19 °: R_avg_opp_SIG_STR_pct: 0.010487367902637873
20 °: R_avg_opp_GROUND_att: 0.010399862628803784
21 °: R_avg_opp_BODY_landed: 0.010264401486569062
22 °: B_avg_CLINCH_landed: 0.01021237619837

#### Criando um dataframe com os dados acima:

In [15]:
data={'Fator':X_train.columns ,'Correlação':model.feature_importances_,}
Fator_por_corr=pd.DataFrame(data)
Fator_por_corr=Fator_por_corr.sort_values(by='Correlação', ascending=False)
Fator_por_corr['Grau de Importância']=lista_j
Fator_por_corr = Fator_por_corr.set_index('Grau de Importância')
Fator_por_corr.head()

,Fator,Correlação
Grau de Importância,,
1°,R_avg_opp_TOTAL_STR_landed,0.015669
2°,R_avg_opp_CLINCH_landed,0.015045
3°,R_age,0.015044
4°,R_total_rounds_fought,0.015024
5°,B_avg_BODY_landed,0.013803


#### Com base na correlação, vamos definir a relevância:

In [16]:
def relevancia(df,coluna_nome,coluna_correlacao,acuracia):
    inuteis = []
    uteis = []
    for index,row in df.iterrows():
        if row[coluna_correlacao] >= -acuracia and row[coluna_correlacao] <= acuracia:
            inuteis.append(row[coluna_nome])
        else:
            uteis.append(row[coluna_nome])
    return uteis

In [17]:
uteis = relevancia(Fator_por_corr,'Fator','Correlação',0.01)

In [18]:
data_heavy_util_relevante = data_heavy_util.loc[:,uteis]
data_heavy_util_relevante.head()

,R_avg_opp_TOTAL_STR_landed,R_avg_opp_CLINCH_landed,R_age,R_total_rounds_fought,B_avg_BODY_landed,B_avg_BODY_att,R_avg_opp_HEAD_landed,R_avg_opp_CLINCH_att,B_avg_HEAD_att,R_avg_opp_GROUND_landed,...,R_losses,B_avg_opp_SIG_STR_pct,R_wins,R_avg_opp_SIG_STR_pct,R_avg_opp_GROUND_att,R_avg_opp_BODY_landed,B_avg_CLINCH_landed,R_avg_SIG_STR_pct,B_avg_CLINCH_att,R_avg_BODY_att
4,27.750000,3.500000,26.0,7,14.500,17.00,14.000000,4.500000,184.5,2.750000,...,1,0.4300,3,0.397500,7.750000,4.750000,2.000,0.545000,2.500,7.750000
60,42.333333,5.333333,32.0,8,8.000,8.00,12.333333,8.000000,131.0,0.666667,...,1,0.5300,2,0.416667,1.333333,7.666667,13.000,0.510000,15.000,2.000000
63,16.000000,1.000000,30.0,2,3.500,5.00,12.000000,2.000000,15.5,3.000000,...,1,0.4300,0,0.400000,3.000000,0.000000,3.500,0.450000,5.500,5.000000
69,34.821429,3.035714,40.0,59,21.000,35.00,15.142857,4.964286,169.0,5.357143,...,12,0.4400,16,0.389286,7.964286,3.750000,30.000,0.453929,38.000,6.607143
75,28.533333,4.200000,38.0,29,3.875,4.25,13.333333,6.466667,28.5,3.466667,...,6,0.4775,9,0.437333,4.133333,3.533333,4.125,0.688667,5.375,13.466667


___
## Verificando a performance


In [19]:
X_train.columns

Index(['title_bout', 'no_of_rounds', 'B_current_lose_streak',
       'B_current_win_streak', 'B_draw', 'B_avg_BODY_att', 'B_avg_BODY_landed',
       'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att',
       ...
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'B_age', 'R_age'],
      dtype='object', length=136)

In [20]:
model.feature_importances_

array([0.00037982, 0.0008378 , 0.00198755, 0.00448719, 0.        ,
       0.01322413, 0.0138026 , 0.01008011, 0.01021238, 0.0087328 ,
       0.00796997, 0.00882968, 0.00836895, 0.01259502, 0.00959801,
       0.00548468, 0.00942306, 0.00854325, 0.00697032, 0.00478324,
       0.0106019 , 0.00954819, 0.0079553 , 0.00416357, 0.01143743,
       0.00673843, 0.00649148, 0.0107663 , 0.00992713, 0.00509203,
       0.00503317, 0.00712215, 0.0068983 , 0.00851778, 0.00804232,
       0.00736991, 0.00763647, 0.00787396, 0.00826296, 0.00833778,
       0.00742492, 0.00454079, 0.00816121, 0.00735323, 0.00608274,
       0.00324957, 0.00811884, 0.00726743, 0.01056398, 0.00470418,
       0.00962376, 0.00620242, 0.00765156, 0.00767997, 0.00830097,
       0.00638697, 0.00931229, 0.00166128, 0.00040887, 0.00176182,
       0.00167487, 0.00461331, 0.00141758, 0.00060273, 0.00418817,
       0.00547292, 0.00754578, 0.00745844, 0.00366894, 0.00394249,
       0.        , 0.01007129, 0.00876238, 0.01137498, 0.01060

In [21]:
y_pred = model.predict(X_test)

In [22]:
print(accuracy_score(y_test, y_pred))

0.5757575757575758


In [23]:
y_pred

array([1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.])

In [24]:
y_test.value_counts(True)

1.0    0.590909
0.0    0.393939
2.0    0.015152
Name: Winner, dtype: float64

In [25]:
data_heavy_num.shape

(264, 137)

### Conclusão desse teste:

Tendo um acurácia de 0.6667 não é um bom resultado, visto que o modelo praticamente sempre tem como resultado o vermelho como vencedor, e como a probabilidade do lutador vermelho ganhar é de 0.6212112 ele praticamente só acerta os que o vencedor é o vermelho e erra os que o azul é o vencedor.

Assim é possível concluir que é necessário desconsiderar algumas variáveis para tentar melhorar a acurácia.

___
## Montando a Regressão Logística


## !Ainda é necessário tirar os fatores que foram julgados ter um baixo grau de importância pelo teste anterior!

In [26]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [27]:
Y = data_heavy_num["Winner"]
data_heavy_sem_Winner=data_heavy_num.drop('Winner',axis=1) 
X=data_heavy_sem_Winner
#np.asarray(X)

In [28]:
result = preparo(X,Y)
result.summary()

C:\Users\Maulem\Spyder\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Winner   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     1.305
Date:                Mon, 18 Nov 2019   Prob (F-statistic):             0.0635
Time:                        22:19:35   Log-Likelihood:                -100.66
No. Observations:                 264   AIC:                             439.3
Df Residuals:                     145   BIC:                             864.9
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            4.6535      2.927      1.590      0.114      -1.131      10.438
title_bout                      -0.0643      0.192     -0.334      0.739      -0.445       0.316
no_of_rounds                     0.0958      0.062      1.533      0.127      -0.028       0.219
B_current_lose_streak           -0.2204      0.073     -3.002      0.003      -0.366      -0.075
B_current_win_streak            -0.0361      0.037     -0.983      0.327      -0.109       0.037
B_draw                        1.562e-13   4.62e-14      3.377      0.001    6.48e-14    2.48e-13
B_avg_BODY_att                  -0.0701      0.040     -1.752      0.082      -0.149       0.009
B_avg_BODY_landed                0.0788      0.049      1.623      0.107      -0.017       0.175
B_avg_CLINCH_att                 0.0259      0.025      1.025      0.307      -0.024       0.076
B_avg_CLINCH_landed             -0.0234      0.035     -0.669      0.504      -0.092       0.046
B_avg_DISTANCE_att              -0.0151      0.014     -1.059      0.291      -0.043       0.013
B_avg_DISTANCE_landed            0.0141      0.019      0.727      0.469      -0.024       0.053
B_avg_GROUND_att                -0.0245      0.023     -1.059      0.291      -0.070       0.021
B_avg_GROUND_landed              0.0242      0.032      0.764      0.446      -0.038       0.087
B_avg_HEAD_att                  -0.0283      0.022     -1.283      0.201      -0.072       0.015
B_avg_HEAD_landed                0.0352      0.028      1.266      0.208      -0.020       0.090
B_avg_KD                         0.3686      0.193      1.906      0.059      -0.014       0.751
B_avg_LEG_att                    0.0846      0.059      1.436      0.153      -0.032       0.201
B_avg_LEG_landed                -0.0990      0.071     -1.386      0.168      -0.240       0.042
B_avg_PASS                       0.1028      0.075      1.375      0.171      -0.045       0.251
B_avg_REV                        0.7377      0.295      2.497      0.014       0.154       1.322
B_avg_SIG_STR_att               -0.0137      0.023     -0.606      0.546      -0.059       0.031
B_avg_SIG_STR_landed             0.0150      0.027      0.551      0.583      -0.039       0.069
B_avg_SIG_STR_pct               -1.5196      0.575     -2.644      0.009      -2.656      -0.384
B_avg_SUB_ATT                   -0.1195      0.137     -0.871      0.385      -0.391       0.152
B_avg_TD_att                    -0.0131      0.048     -0.274      0.784      -0.107       0.081
B_avg_TD_landed                 -0.0729      0.104     -0.701      0.484      -0.278       0.133
B_avg_TD_pct                     0.3372      0.275      1.228      0.222      -0.206       0.880
B_avg_TOTAL_STR_att              0.0405      0.026      1.568      0.119    

In [29]:
'''data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)'''

"data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]\ndata_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)\ndata_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]\ndata_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)"

In [30]:
'''blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head()'''

'blue_winners_correlation = data_heavy_blue_winners.corr()\nblue_winners_correlation.head()'

In [31]:
'''red_winners_correlation = data_heavy_red_winners.corr()
red_winners_correlation.head()'''

'red_winners_correlation = data_heavy_red_winners.corr()\nred_winners_correlation.head()'

___
## Concluindo

   ___
## Aperfeiçoamento:


   ___
## Referências

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

[Como funciona o Random Forest](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)

[Referencia do Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a Regressão Linear](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)



In [32]:
#criando um novo dataframe
lista_locB = ["B_fighter", "weight_class","B_Height_cms","B_Reach_cms","B_Weight_lbs","B_age"]
lista_locR = ["R_fighter", "weight_class","R_Height_cms","R_Reach_cms","R_Weight_lbs","R_age"]
data_red = data2.loc[: , lista_locR]
data_blue = data2.loc[: , lista_locB]

data_blue.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]
data_red.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]

data_red

,Fighter,Weight_class,Height_cms,Reach_cms,Weight_lbs,Age
0,Henry Cejudo,Bantamweight,162.56,162.56,135.0,32.0
1,Valentina Shevchenko,Women's Flyweight,165.10,167.64,125.0,31.0
2,Tony Ferguson,Lightweight,180.34,193.04,155.0,35.0
3,Jimmie Rivera,Bantamweight,162.56,172.72,135.0,29.0
4,Tai Tuivasa,Heavyweight,187.96,190.50,264.0,26.0
5,Tatiana Suarez,Women's Strawweight,165.10,167.64,115.0,28.0
6,Aljamain Sterling,Bantamweight,170.18,180.34,135.0,29.0
7,Karolina Kowalkiewicz,Women's Strawweight,160.02,162.56,115.0,33.0
8,Ricardo Lamas,Featherweight,172.72,180.34,145.0,37.0
9,Yan Xiaonan,Women's Strawweight,165.10,160.02,115.0,29.0


In [35]:
newdataR = pd.DataFrame()


namelistR = []

for n in range(0,5143):
    if data_red.loc[n,:]["Fighter"] not in namelistR:
        namelistR.append(data_red.loc[n,:]["Fighter"])
        newdataR = pd.concat([newdataR,data_red.loc[n,:]], axis=1, join='outer')

newdataR = newdataR.transpose()

newdataR 

C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,Age,Fighter,Height_cms,Reach_cms,Weight_class,Weight_lbs
0,32,Henry Cejudo,162.56,162.56,Bantamweight,135
1,31,Valentina Shevchenko,165.1,167.64,Women's Flyweight,125
2,35,Tony Ferguson,180.34,193.04,Lightweight,155
3,29,Jimmie Rivera,162.56,172.72,Bantamweight,135
4,26,Tai Tuivasa,187.96,190.5,Heavyweight,264
5,28,Tatiana Suarez,165.1,167.64,Women's Strawweight,115
6,29,Aljamain Sterling,170.18,180.34,Bantamweight,135
7,33,Karolina Kowalkiewicz,160.02,162.56,Women's Strawweight,115
8,37,Ricardo Lamas,172.72,180.34,Featherweight,145
9,29,Yan Xiaonan,165.1,160.02,Women's Strawweight,115


In [36]:
newdataB = pd.DataFrame()


namelistB = []

for n in range(0,5143):
    if data_blue.loc[n,:]["Fighter"] not in namelistB:
        namelistB.append(data_blue.loc[n,:]["Fighter"])
        newdataB = pd.concat([newdataB,data_blue.loc[n,:]], axis=1, join='outer')

newdataB = newdataB.transpose()

newdataB

C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,Age,Fighter,Height_cms,Reach_cms,Weight_class,Weight_lbs
0,31,Marlon Moraes,167.64,170.18,Bantamweight,135
1,32,Jessica Eye,167.64,167.64,Women's Flyweight,125
2,36,Donald Cerrone,185.42,185.42,Lightweight,155
3,26,Petr Yan,170.18,170.18,Bantamweight,135
4,32,Blagoy Ivanov,180.34,185.42,Heavyweight,250
5,33,Nina Ansaroff,165.1,162.56,Women's Strawweight,115
6,32,Pedro Munhoz,167.64,165.1,Bantamweight,135
7,25,Alexa Grasso,165.1,167.64,Women's Strawweight,115
8,31,Calvin Kattar,180.34,182.88,Featherweight,145
9,34,Angela Hill,160.02,162.56,Women's Strawweight,115


In [37]:
newdata = pd.concat([newdataB, newdataR], axis=0, join='outer', ignore_index=False, keys=None,levels=None, names=None, verify_integrity=False, copy=True)
newdata = newdata.set_index("Fighter")

newdata

,Age,Height_cms,Reach_cms,Weight_class,Weight_lbs
Fighter,,,,,
Marlon Moraes,31,167.64,170.18,Bantamweight,135
Jessica Eye,32,167.64,167.64,Women's Flyweight,125
Donald Cerrone,36,185.42,185.42,Lightweight,155
Petr Yan,26,170.18,170.18,Bantamweight,135
Blagoy Ivanov,32,180.34,185.42,Heavyweight,250
Nina Ansaroff,33,165.1,162.56,Women's Strawweight,115
Pedro Munhoz,32,167.64,165.1,Bantamweight,135
Alexa Grasso,25,165.1,167.64,Women's Strawweight,115
Calvin Kattar,31,180.34,182.88,Featherweight,145
